In [1]:
import pandas as pd
import datetime
import yfinance as yf
from decimal import Decimal

In [2]:
today = datetime.datetime.now().strftime('%Y-%m-%d')

In [3]:
ibov = yf.download('^BVSP', start='2024-08-01', end=today)
usd_brl = yf.download('BRL=X', start='2010-01-01', end=today)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
def consulta_bc(codigo_bcb):
  url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  return df

In [5]:
ipca = consulta_bc(433)
igpm = consulta_bc(189)
selic_meta = consulta_bc(432)
reservas_internacionais = consulta_bc(13621)

In [6]:
df_economicos = pd.concat([ipca, igpm, selic_meta, reservas_internacionais], axis=1)

# Renomeando as colunas de forma adequada
df_economicos.columns = ['IPCA', 'IGPM', 'SELIC Meta', 'Reservas Internacionais']

# Resetando o índice para ter a coluna de 'Data'
df_economicos.reset_index(inplace=True)

# Renomeando a coluna de datas para "Data"
df_economicos.rename(columns={'index': 'Data'}, inplace=True)

# Substituir valores NaN por 0
df_economicos.fillna(0, inplace=True)

In [7]:
df_economicos .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9730 entries, 0 to 9729
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   data                     9730 non-null   datetime64[ns]
 1   IPCA                     9730 non-null   float64       
 2   IGPM                     9730 non-null   float64       
 3   SELIC Meta               9730 non-null   float64       
 4   Reservas Internacionais  9730 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 380.2 KB


In [8]:
# Colunas que deseja converter de float para decimal
colunas_float = ['IPCA','IGPM','SELIC Meta','Reservas Internacionais']

def formatar_decimal(valor):
    return str(valor).replace('.', ',') if valor is not None else ''

# Aplicar a formatação para cada coluna de float
for coluna in colunas_float:
    df_economicos[coluna] = df_economicos[coluna].apply(formatar_decimal)


In [9]:
df_economicos.head()

,data,IPCA,IGPM,SELIC Meta,Reservas Internacionais
0,1980-02-01,"4,62","0,0","0,0","0,0"
1,1980-03-01,"6,04","0,0","0,0","0,0"
2,1980-04-01,"5,29","0,0","0,0","0,0"
3,1980-05-01,"5,7","0,0","0,0","0,0"
4,1980-06-01,"5,31","0,0","0,0","0,0"


In [10]:
# Salvar o DataFrame em um arquivo CSV, usando vírgula como delimitador
df_economicos.to_csv('dados_economicos.csv', sep=';', index=False, decimal=',')

In [11]:
df_economicos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9730 entries, 0 to 9729
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   data                     9730 non-null   datetime64[ns]
 1   IPCA                     9730 non-null   object        
 2   IGPM                     9730 non-null   object        
 3   SELIC Meta               9730 non-null   object        
 4   Reservas Internacionais  9730 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 380.2+ KB
